In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import warnings
import ast
warnings.filterwarnings('ignore')

In [2]:
# Cargar los datos de juegos (descripciones)
df_juegos = pd.read_csv('../data_extraction/boardgames_10000_juegos.csv')

In [3]:
df_juegos.columns

Index(['BGGId', 'Name', 'Year_Published', 'Description', 'Min_Players',
       'Max_Players', 'Min_Playtime', 'Max_Playtime', 'Average_Rating',
       'Bayesian_Average_Rating', 'Number_of_Ratings', 'Mechanics',
       'Categories'],
      dtype='object')

In [4]:
# Obtener la descripción de los juegos
df_descripciones = df_juegos[['BGGId','Name','Description', 'Mechanics', 'Categories', 'Average_Rating', 'Bayesian_Average_Rating']]  

In [5]:
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    10000 non-null  int64  
 1   Name                     10000 non-null  object 
 2   Description              9992 non-null   object 
 3   Mechanics                10000 non-null  object 
 4   Categories               10000 non-null  object 
 5   Average_Rating           10000 non-null  float64
 6   Bayesian_Average_Rating  10000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 547.0+ KB


In [6]:
df_descripciones.dropna(inplace=True)
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    9992 non-null   int64  
 1   Name                     9992 non-null   object 
 2   Description              9992 non-null   object 
 3   Mechanics                9992 non-null   object 
 4   Categories               9992 non-null   object 
 5   Average_Rating           9992 non-null   float64
 6   Bayesian_Average_Rating  9992 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 624.5+ KB


In [7]:
df_descripciones.sample(5)

,BGGId,Name,Description,Mechanics,Categories,Average_Rating,Bayesian_Average_Rating
3775,130695,Shantytown Shakedown,From Introduction:&#10;&#10;Encounter Summary:...,[],[],0.00000,0.00000
1632,212851,Strands of Gloom,A One-Round Living Greyhawk Theocracy of the P...,[],[],0.00000,0.00000
1011,289328,Rebel Times (Issue 144 - Sep 2019),Rebel Times&#10;Numer 144&#10;Wrzesie&#197;&#1...,[],[],0.00000,0.00000
5667,26474,Cold War Commander: Fast-Play Tabletop Wargame...,Cold War Commander is a wargame that allows yo...,"['Dice Rolling', 'Line of Sight', 'Measurement...","['Modern Warfare', 'Book', 'Korean War', 'Mini...",7.35667,5.55133
6835,233436,Yet Another Nameless Dungeon,Publisher's blurb:&#10;&#10;I think it says a ...,[],[],0.00000,0.00000


In [8]:
# Asegúrate de descargar stopwords y wordnet de nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Función mejorada de preprocesamiento de texto
def preprocesar_descripcion(texto):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Eliminar entidades HTML y otros caracteres no deseados
    texto = re.sub(r'&#?\w+;', ' ', texto)  # Reemplazar entidades HTML como &#10; 
    texto = re.sub(r'\W+', ' ', texto)  # Eliminar caracteres no alfanuméricos
    texto = re.sub(r'\d+', '', texto)  # Eliminar números
    
    # Convertir a minúsculas, eliminar stopwords y lematizar (solo si el texto es válido)
    if isinstance(texto, str):
        palabras = [lemmatizer.lemmatize(palabra) for palabra in texto.lower().split() if palabra not in stop_words]
        return " ".join(palabras)
    else:
        return ""  # Devolver cadena vacía si no es un texto válido



[nltk_data] Downloading package stopwords to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Laura
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Convertir cadenas de texto que parecen listas a listas reales
df_descripciones['Mechanics'] = df_descripciones['Mechanics'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_descripciones['Categories'] = df_descripciones['Categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Verificar si las listas están correctamente formateadas
print(df_juegos[['Mechanics', 'Categories']].head())

                                           Mechanics  \
0                  ['Action Points', 'Dice Rolling']   
1                                                 []   
2                                                 []   
3  ['Dice Rolling', 'Grid Movement', 'Hexagon Gri...   
4                                                 []   

                                          Categories  
0  ['Ancient', 'Book', 'Medieval', 'Miniatures', ...  
1                                                 []  
2                                                 []  
3                        ['Wargame', 'World War II']  
4                                                 []  


In [10]:
# Convertir listas a cadenas de texto
df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x) if isinstance(x, list) and len(x) > 0 else "")
df_descripciones['Categorias'] = df_descripciones['Categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) and len(x) > 0 else "")

# Combinar la descripción limpia con las mecánicas y categorías
df_descripciones['Descripcion_completa'] = df_descripciones['Description'] + " " + df_descripciones['Mecanicas'] + " " + df_descripciones['Categorias']

In [11]:
# Aplicar el preprocesamiento mejorado a las descripciones
df_descripciones['Descripcion_completa'] = df_descripciones['Descripcion_completa'].apply(preprocesar_descripcion)

In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Entrenar Doc2Vec usando los índices del DataFrame como etiquetas
documents = [TaggedDocument(doc.split(), [idx]) for idx, doc in zip(df_descripciones.index, df_descripciones['Descripcion_completa'])]




In [13]:
# Entrenar el modelo Doc2Vec
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4, seed = 42)

In [14]:
# Función para recomendar juegos usando Doc2Vec y similaridad del coseno
def recomendar_juegos_doc2vec(prompt, df_descripciones, model, top_n=5):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos
    vectors = [model.dv[i] for i in range(len(df_descripciones))]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Obtener los índices de los juegos con mayor similaridad
    indices_recomendados = cosine_similarities.argsort()[-top_n:][::-1]

    # Mostrar los juegos recomendados
    print("Juegos recomendados:")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad: {cosine_similarities[index]:.4f})")

In [17]:
# Prompt del usuario
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'
# Ejecutar la recomendación usando Doc2Vec
recomendar_juegos_doc2vec(prompt_usuario, df_descripciones, model, top_n=5)

Juegos recomendados:
Married!  The Game of Marriage and Money (Similaridad: 0.6265)
Worldbuilding Monthly (Issue 5 / August 2017) - Mythologies (Similaridad: 0.5651)
Urban Myth: Funsize Crime Edition (Similaridad: 0.5629)
Król Popiel (Similaridad: 0.5602)
Roll For It! (Similaridad: 0.5505)


In [19]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_descripciones, model, top_n=5, umbral_similaridad=0.1):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[idx] for idx in df_descripciones.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_descripciones.loc[df_descripciones.index.isin(df_descripciones.index), 'Average_Rating'].fillna(0).values

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones = valoraciones[:min_len]

    # Crear una métrica combinada (similaridad * valoraciones)
    similaridad_ponderada = cosine_similarities * valoraciones

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print("Juegos recomendados (con valoraciones y similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")



In [20]:
prompt_usuario = 'I am looking for a game that is great for solo play and lasts about 30 minutes. I love games with a rich storytelling experience and immersive themes.'

In [26]:
# Ejecutar la recomendación usando Doc2Vec con valoraciones ponderadas y umbral
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1)

Juegos recomendados (con valoraciones y similaridad del coseno):
The Murder of Mr. Crow (Similaridad ponderada: 5.1135, Similaridad del coseno: 0.5113, Valoración: 10.00)
Whirred PLAY (Similaridad ponderada: 4.5955, Similaridad del coseno: 0.4595, Valoración: 10.00)
Forever Young (Similaridad ponderada: 4.2956, Similaridad del coseno: 0.4296, Valoración: 10.00)
FREE-O Card Game (Similaridad ponderada: 3.7163, Similaridad del coseno: 0.3716, Valoración: 10.00)
Social! (Similaridad ponderada: 3.5278, Similaridad del coseno: 0.3920, Valoración: 9.00)


In [27]:
def recomendar_juegos_doc2vec_con_valoraciones(prompt, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7):
    # Preprocesar el prompt y vectorizarlo usando Doc2Vec
    prompt_vector = model.infer_vector(prompt.split())

    # Obtener los vectores de todos los juegos usando los índices correctos del DataFrame
    vectors = [model.dv[idx] for idx in df_descripciones.index]

    # Calcular la similaridad del coseno entre el prompt y los juegos
    cosine_similarities = cosine_similarity([prompt_vector], vectors).flatten()

    # Asegurarse de que las valoraciones coincidan con los juegos vectorizados
    valoraciones = df_descripciones.loc[df_descripciones.index.isin(df_descripciones.index), 'Average_Rating'].fillna(0).values

    # Normalizar las valoraciones a una escala de 0 a 1
    valoraciones_normalizadas = valoraciones / 10  # Suponiendo que la escala máxima es 10

    # Asegurar que los tamaños de cosine_similarities y valoraciones coincidan
    if len(cosine_similarities) != len(valoraciones):
        min_len = min(len(cosine_similarities), len(valoraciones))
        cosine_similarities = cosine_similarities[:min_len]
        valoraciones_normalizadas = valoraciones_normalizadas[:min_len]

    # Crear una métrica ponderada que priorice más la similaridad del coseno
    similaridad_ponderada = (cosine_similarities * w) + (valoraciones_normalizadas * (1 - w))

    # Filtrar recomendaciones con similaridad negativa o muy baja
    indices_validos = [i for i, sim in enumerate(cosine_similarities) if sim > umbral_similaridad]

    # Ordenar los juegos con mayor similaridad ponderada
    indices_recomendados = sorted(indices_validos, key=lambda i: similaridad_ponderada[i], reverse=True)[:top_n]

    # Mostrar los juegos recomendados
    print(f"Juegos recomendados (priorizando similaridad del coseno):")
    for index in indices_recomendados:
        print(f"{df_descripciones.iloc[index]['Name']} (Similaridad ponderada: {similaridad_ponderada[index]:.4f}, "
              f"Similaridad del coseno: {cosine_similarities[index]:.4f}, "
              f"Valoración: {valoraciones[index]:.2f})")


In [29]:
# Ejecutar la recomendación priorizando la similaridad del coseno (peso w = 0.7)
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7)

Juegos recomendados (priorizando similaridad del coseno):
The Murder of Mr. Crow (Similaridad ponderada: 0.6467, Similaridad del coseno: 0.4953, Valoración: 10.00)
Social! (Similaridad ponderada: 0.6379, Similaridad del coseno: 0.5255, Valoración: 9.00)
Whirred PLAY (Similaridad ponderada: 0.6336, Similaridad del coseno: 0.4766, Valoración: 10.00)
Punctuation Pirates and Their Grammar Galleons (Similaridad ponderada: 0.6152, Similaridad del coseno: 0.4503, Valoración: 10.00)
Front Page Sports: Baseball Pro '98 (Similaridad ponderada: 0.5988, Similaridad del coseno: 0.4484, Valoración: 9.50)
